<a href="https://colab.research.google.com/github/jorgemarquesferreirajunior/YOLO/blob/main/DeteccaoOpencvYOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detectando objetos com YOLO v4 - implementação com OpenCV

## Etapa 1 - Importando as bibliotecas

In [ ]:
import cv2
print(cv2.__version__)

In [ ]:
#!pip install opencv-python==4.4.0.40

In [ ]:
import cv2
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import zipfile
print(cv2.__version__)

## Etapa 2 - Conectando com o Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

## Etapa 3 - Carregando os arquivos do modelo treinado

In [ ]:
path = '/content/Drive/MyDrive/ColabNotebooks/Curso-DeteccaoDeObjetosYolo/DataBase-recursos-curso-Yolo/YOLO/modelo_YOLOv4.zip'
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
labels_path = os.path.sep.join(['/content/cfg', 'coco.names'])
labels_path

In [ ]:
LABELS = open(labels_path).read().strip().split('\n')
print(LABELS)

In [ ]:
len(LABELS)

In [ ]:
weights_path = os.path.sep.join(['/content', 'yolov4.weights'])
config_path = os.path.sep.join(['/content/cfg', 'yolov4.cfg'])
weights_path, config_path

In [ ]:
net = cv2.dnn.readNet(config_path, weights_path)

In [ ]:
net

## Etapa 4 - Definindo mais configurações para a detecção

In [ ]:
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype='uint8')
print(COLORS)

In [ ]:
layerNames = net.getLayerNames()
print('Todas as camadas')
print(layerNames)
print('Total de camadas: ' + str(len(layerNames)))

In [ ]:
print('Indices das camadas de saída')
print(net.getUnconnectedOutLayers())
print('Camadas de saída')
ln = net.getUnconnectedOutLayersNames()
print(ln)

## Etapa 5 - Carregando a imagem onde será feita a detecção

In [ ]:
def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(16,10)
  plt.axis('off')
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
parent_dir = os.path.dirname(path)
path, parent_dir

In [ ]:
import subprocess

In [ ]:
images_path =  os.path.join(parent_dir, 'imagens')
result = subprocess.run(['ls', images_path], capture_output=True, text=True)
print(result.stdout)
images_path = [os.path.join(images_path, f) for f in os.listdir(images_path) if f.lower().endswith('.jpg')]
images_path

In [ ]:
for img in images_path:
    mostrar(cv2.imread(img))

imagem = cv2.imread(images_path[0])

In [ ]:
type(imagem)

In [ ]:
print(imagem)

In [ ]:
imagem_cp = imagem.copy()

In [ ]:
imagem.shape

In [ ]:
(H, W) = imagem.shape[:2]
print('Altura: ' + str(H) + '\nLargura: ' + str(W))

## Etapa 6 - Processando a imagem de entrada

In [ ]:
inicio = time.time()

blob = cv2.dnn.blobFromImage(imagem, 1 / 255.0, (416, 416), swapRB = True, crop = False)
net.setInput(blob)
layer_outputs = net.forward(ln)

termino = time.time()
print('YOLO levou {:.2f} segundos'.format(termino - inicio))

## Etapa 7 - Definindo as variáveis

In [ ]:
threshold = 0.5
threshold_NMS = 0.3
caixas = []
confiancas = []
IDclasses = []

## Etapa 8 - Realizando a predição

In [ ]:
len(layer_outputs)

In [ ]:
layer_outputs[0], len(layer_outputs[0])

In [ ]:
layer_outputs[1]

In [ ]:
layer_outputs[2], layer_outputs[2][0], len(layer_outputs[2][0])

In [ ]:
print(LABELS)

In [ ]:
LABELS[16]

In [ ]:
np.argmax(np.array([0.10, 0.80, 0.10]))

In [ ]:
teste = np.array([0.10, 0.80, 0.10])

In [ ]:
teste[1]

In [ ]:
for output in layer_outputs:
  for detection in output:
    scores = detection[5:]
    classeID = np.argmax(scores)
    confianca = scores[classeID]
    if confianca > threshold:
      print('scores: ' + str(scores))
      print('classe mais provável: ' + str(classeID))
      print('confiança: ' + str(confianca))

      caixa = detection[0:4] * np.array([W, H, W, H])
      (centerX, centerY, width, height) = caixa.astype('int')

      x = int(centerX - (width / 2))
      y = int(centerY - (height / 2))

      caixas.append([x, y, int(width), int(height)])
      confiancas.append(float(confianca))
      IDclasses.append(classeID)

In [ ]:
print(caixas), len(caixas)

In [ ]:
print(confiancas), len(confiancas)

In [ ]:
print(IDclasses), len(IDclasses)

## Etapa 9 - Aplicando a Non-Maxima Suppression

In [ ]:
objs = cv2.dnn.NMSBoxes(caixas, confiancas, threshold, threshold_NMS)

In [ ]:
objs

In [ ]:
print(objs.flatten())

In [ ]:
confiancas[7], confiancas[15], confiancas[11]

In [ ]:
confiancas[0]

## Etapa 10 - Mostrando o resultado da detecção na imagem

In [ ]:
if len(objs) > 0:
  for i in objs.flatten():
    (x, y) = (caixas[i][0], caixas[i][1])
    (w, h) = (caixas[i][2], caixas[i][3])

    objeto = imagem_cp[y:y + h, x:x + w]
    cv2_imshow(objeto)

    cor = [int(c) for c in COLORS[IDclasses[i]]]

    cv2.rectangle(imagem, (x, y), (x + w, y + h), cor, 2)
    texto = "{}: {:.4f}".format(LABELS[IDclasses[i]], confiancas[i])
    cv2.putText(imagem, texto, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor, 2)

In [ ]:
mostrar(imagem)

In [ ]:
cv2.imwrite('resultado.jpg', imagem)